In [1]:
import sys, io, os

os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/"

import numpy as np
from libs import myDehankelization, myHankelization, myBTD
import seaborn as sns
import matplotlib.pyplot as plt
import matlab
import pandas as pd
from random import randint
from scipy.io import savemat
from scipy.signal import resample
import ecg_plot
import scipy.fftpack as fftpack
from scipy.stats import kurtosis
from scipy.linalg import norm

/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
'''

Getting df Aldebaran

df_0 = pd.read_pickle('../data/part_0.pkl')
df_1 = pd.read_pickle('../data/part_1.pkl')
df = df_0.append(df_1)
df = df.groupby(['diagnostic', 'db']).head(10)
df.to_pickle(f'../aldebaran/data.pkl')
df.groupby(['diagnostic', 'db']).size()
'''

diagnostic  db                  
AF          WFDB_CPSC2018           10
            WFDB_CPSC2018_2         10
            WFDB_ChapmanShaoxing    10
            WFDB_Ga                 10
            WFDB_PTBXL              10
SR          WFDB_CPSC2018           10
            WFDB_CPSC2018_2         10
            WFDB_ChapmanShaoxing    10
            WFDB_Ga                 10
            WFDB_Ningbo             10
            WFDB_PTBXL              10
dtype: int64

In [2]:
df = pd.read_pickle(f'../aldebaran/data.pkl')
df.groupby(['diagnostic', 'db']).size()

diagnostic  db                  
AF          WFDB_CPSC2018           10
            WFDB_CPSC2018_2         10
            WFDB_ChapmanShaoxing    10
            WFDB_Ga                 10
            WFDB_PTBXL              10
SR          WFDB_CPSC2018           10
            WFDB_CPSC2018_2         10
            WFDB_ChapmanShaoxing    10
            WFDB_Ga                 10
            WFDB_Ningbo             10
            WFDB_PTBXL              10
dtype: int64

In [4]:
for index, row in df.iloc[0:1].iterrows():
    recording = row['data']
    leads, samples = recording.shape
    new_freq = 100
    time_rec = samples/250
    n_samples = int(time_rec*new_freq)
    recording = resample(recording, n_samples, axis=1)

    T = np.zeros((50, 51, leads))
    my_hankelization = myHankelization.initialize()
    for i in range(0, leads):
        vec = matlab.double(recording[i].tolist(), size=recording[i].shape)
        T[:, :, i] = my_hankelization.hankelization(vec)
    my_hankelization.terminate()

    T = matlab.double(T.tolist(), size=T.shape)
    R = matlab.double([6.0], size=(1, 1))
    max_iter = matlab.double([10.0], size=(1, 1))
    multirank = matlab.double([40.0, 40.0, 1.0], size=(1, 3))



    btd = myBTD.initialize()
    output = btd.myBTD(R, multirank, max_iter, T)

    #output = np.array(output)
    btd.terminate()



    path = f'../aldebaran/rank_{R._data[0]}'
    if not os.path.exists(path):
        os.makedirs(path)

    with open(f'{path}/{row.name}.npy', 'wb') as file:
        np.save(file, output)




       fval             relfval          relstep          delta        rho
       =1/2*norm(F)^2   TolFun = 1e-12   TolX = 1e-06

   0:  3.40400905e+09 |
   1:  3.25897730e+09 | 4.26061568e-02 | 3.00000000e-01 | 5.9529e+01 | 7.0482e+00
   2:  2.22598982e+09 | 3.03462025e-01 | 1.43452693e-01 | 2.9764e+01 | 1.8552e+00
   3:  1.44093372e+09 | 2.30626915e-01 | 6.99428218e-02 | 1.4937e+01 | 1.2063e+00
   4:  1.00142013e+09 | 1.29116457e-01 | 1.36366165e-01 | 2.9874e+01 | 9.8303e-01
   5:  7.61094857e+08 | 7.06006562e-02 | 2.61397131e-01 | 5.9749e+01 | 1.0047e+00
   6:  4.36532594e+08 | 9.53470623e-02 | 5.05907888e-01 | 1.1950e+02 | 1.1338e+00
   7:  1.52535685e+08 | 8.34301277e-02 | 8.55561389e-01 | 2.3899e+02 | 1.2079e+00
   8:  1.24880197e+08 | 8.12438750e-03 | 2.97821835e-01 | 5.8324e+01 | 4.9404e-01
   9:  4.89109031e+07 | 2.23175945e-02 | 2.57121322e-01 | 1.1665e+02 | 9.6924e-01
  10:  2.91528650e+07 | 5.80434358e-03 | 1.18382570e-01 | 5.8324e+01 | 9.7480e-01

Maximum number of iterat

In [5]:
output

{'output': [[matlab.double([[-1.1707831225288163,-0.7740139591026487,0.18328323765441792,-0.2784605593029458,-0.2529136682193178,0.5466559444038557,-0.31261532999249425,0.024292850937263535,0.1346882525025833,-0.25474580213509646,-0.4613282530386046,-0.9025062238178171,-0.6493233401291998,-0.08112611895979777,0.013110976835854132,-0.1027005383552412,-0.32338620440801685,-0.2748870572324382,0.6354311519675929,0.08759085981299618,0.5197697828897437,0.1473407836007813,-0.21481816931784187,0.1108147555570277,0.37565740159161803,0.6762865293376523,0.6969525276402795,0.4310246983863909,-0.42145841493118386,0.7087651768314158,0.5288195268997293,-0.23589680240966943,0.7539549490430352,1.9068778393226484,1.2679915390169332,0.37002820534956465,0.3867165684538514,-0.556017582670321,1.0297618958064407,-0.9254592105779381],[-0.9109173223365516,-0.5296313808642299,0.057826340031020135,0.17816586201822726,-0.6700079678458938,0.4030284375122484,-0.49372876575151065,-0.1960913020660237,-0.0965479664780

In [ ]:
def get_sources(filename):
    with open(f"tensor_data/6d2698_indexes/{filename}", 'rb') as f:
            output = np.load(f, allow_pickle=True)
    output = np.array(output)
    sources = np.empty((0, 100), int)
    contrib_sources = np.empty((0, 12), int)
    my_dehankelization = myDehankelization.initialize()
    for i in range(0, len(output)):
        a = np.array(output[i][0])
        b = np.array(output[i][1]).T
        c = np.array(output[i][2]).T
        contrib_sources = np.append(contrib_sources, c, axis=0)
        mixing_matrix = np.dot(a, b)
        m_hankel = matlab.double(mixing_matrix.tolist(), size=mixing_matrix.shape)
        source = my_dehankelization.dehankelization(m_hankel)
        sources = np.append(sources, np.array(source).T, axis=0)
    my_dehankelization.terminate()
    return sources, contrib_sources


def get_likely_atrial(sources, contrib_sources):
    x = np.linspace(0, 1, 100)
    potentials_aa = np.empty((0, 100), int)
    for i, source in enumerate(sources):
        fourier = np.fft.fft(source)
        fourier = fourier[:int(len(source)/2)]
        frequencies = np.fft.fftfreq(len(source), d=x[1]-x[0])[:int(len(source)/2)]
        frequencies = frequencies[:int(len(source)/2)]
        freq = frequencies[np.argmax(np.abs(fourier))]
        contrib_source = contrib_sources[i, :]
        contrib_source_V1 = contrib_source[1]
        power_contrib = (norm(contrib_source_V1*source)**2)/100
        if freq > 3 and freq < 9 and power_contrib > 0.0001:
            potentials_aa = np.append(potentials_aa, np.array([source]), axis=0)
    return potentials_aa


def get_atrial_source(potentials_aa):
    kurt_pot_aa = [*map(lambda p_aa: kurtosis(p_aa), potentials_aa)]
    atrial_source = potentials_aa[kurt_pot_aa.index(max(kurt_pot_aa))]
    return atrial_source
    
    
def get_df_atrial():
    files = os.listdir('tensor_data/6d2698_indexes')
    rows = list()
    for filename in files:
        sources, contrib_sources = get_sources(filename)
        # potentials_aa = get_likely_atrial(sources, contrib_sources)
        #if potentials_aa.size != 0:
        #    atrial_source = get_atrial_source(potentials_aa)
        label = df.loc[int(filename.split('.')[0])].diagnostic
        rows.append({
              'id': int(filename.split('.')[0]),
              'data': sources,
              'label': label
        })
    return pd.DataFrame(rows)

df_atrial = get_df_atrial()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

df_atrial = pd.read_pickle('df_atrial.pkl')
label_encoder = LabelEncoder()
